# Tutorial 02: Módulo Merger - Combinación de Módulos ENAHO

Este tutorial cubre el uso del módulo **merger** de enahopy para:
- Combinar múltiples módulos ENAHO a nivel hogar
- Combinar módulos a nivel persona
- Validar la calidad de los merges
- Gestionar conflictos y valores faltantes

## Ejemplos que cubriremos

1. **Merge a Nivel Hogar**: Módulo 34 (Sumaria) + Módulo 01 (Características de Vivienda)
2. **Merge a Nivel Persona**: Módulo 05 (Empleo) + Módulo 02 (Características de Miembros)
3. **Merge Multi-nivel**: Combinar datos de hogar y persona

In [3]:
import enahopy
from enahopy.loader import ENAHODataDownloader
from enahopy.loader.io import ENAHOLocalReader
from enahopy.merger import ENAHOGeoMerger, ModuleMergeConfig, ModuleMergeLevel
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print(f"enahopy versión: {enahopy.__version__}")

enahopy versión: 0.5.1
enahopy versión: 0.5.1
enahopy versión: 0.5.1


## 1. Preparación: Cargar Datos

### 1.1 Configurar Descargador y Lector

In [4]:
# Configurar año de análisis
year = 2022

# Inicializar descargador
downloader = ENAHODataDownloader(verbose=False)  # Silencioso para claridad

print(f"✅ Configuración lista para año {year}")

✅ Configuración lista para año 2022


### 1.2 Cargar Datos a Nivel Hogar

In [5]:
import glob
import os

# Descargar Módulo 34 (Sumaria)
#print("📥 Descargando Módulo 34 (Sumaria)...")
#downloader.download(modules=["34"], years=[str(year)], output_dir=".enaho_cache", decompress=True)

# Encontrar y leer archivo
dta_files_34 = glob.glob(f".enaho_cache/modulo_34_{year}/*.dta")
if dta_files_34:
    file_path_34 = dta_files_34[0]
    reader_34 = ENAHOLocalReader(file_path=file_path_34, verbose=False)
    df_sumaria, _ = reader_34.read_data(columns=[
        'conglome', 'vivienda', 'hogar',  # Identificadores
        'ubigeo', 'dominio', 'estrato',   # Ubicación
        'mieperho',   # Miembros del hogar
        'gashog2d',   # Gasto total
        'inghog2d',   # Ingreso total
        'pobreza',    # Condición de pobreza
        'facpob07'    # Factor de expansión
    ])
    print(f"✅ Módulo 34 cargado: {len(df_sumaria):,} hogares")
else:
    raise FileNotFoundError("No se encontró archivo .dta para módulo 34")

# Descargar Módulo 01 (Características de Vivienda)
#print("\n📥 Descargando Módulo 01 (Vivienda y Hogar)...")
#downloader.download(modules=["01"], years=[str(year)], output_dir=".enaho_cache", decompress=True)

# Encontrar y leer archivo
dta_files_01 = glob.glob(f".enaho_cache/modulo_01_{year}/*.dta")
if dta_files_01:
    file_path_01 = dta_files_01[0]
    reader_01 = ENAHOLocalReader(file_path=file_path_01, verbose=False)
    df_vivienda, _ = reader_01.read_data(columns=[
        'conglome', 'vivienda', 'hogar',  # Identificadores
        'result',     # Resultado de la encuesta
        'p101',       # Tipo de vivienda
        'p102',       # Material de paredes
        'p103',       # Material de pisos
        'p103a',      # Material de techo
        'p110',       # Agua potable
        'p111',       # Servicios higiénicos
        'p112',       # Alumbrado eléctrico
        'p113',       # Número de habitaciones
        'p1141',      # Celular en el hogar
        'p1142',      # Teléfono fijo
        'p1143',      # Internet
        'p1144',      # TV por cable
    ])
    print(f"✅ Módulo 01 cargado: {len(df_vivienda):,} hogares")
else:
    raise FileNotFoundError("No se encontró archivo .dta para módulo 01")

# Mostrar primeras filas
print("\n📊 Muestra de datos:")
print("\nMódulo 34 (Sumaria):")
display(df_sumaria.head(3))

print("\nMódulo 01 (Vivienda):")
display(df_vivienda.head(3))

✅ Módulo 34 cargado: 34,213 hogares
✅ Módulo 01 cargado: 44,122 hogares

📊 Muestra de datos:

Módulo 34 (Sumaria):


,conglome,vivienda,hogar,ubigeo,dominio,estrato,mieperho,gashog2d,inghog2d,pobreza
0,005030,008,11,010201,selva,de 20 000 a 49 999 habitantes,2,36949.523438,34606.441406,no pobre
1,005030,017,11,010201,selva,de 20 000 a 49 999 habitantes,2,48161.250000,82028.218750,no pobre
2,005030,033,11,010201,selva,de 20 000 a 49 999 habitantes,4,27575.878906,31791.029297,no pobre



Módulo 01 (Vivienda):


,conglome,vivienda,hogar,result,p101,p102,p103,p103a,p110,p1141,p1142,p1143,p1144
0,005007,003,11,completa,casa independiente,adobe,"losetas, terrazos o similares",madera,"red publica, dentro de la vivienda",pase,telefono celular,pase,conexion a internet
1,005007,012,11,completa,"vivienda en casa de vecindad (callejon, solar ...",tapia,cemento,madera,"red publica, fuera de la vivienda pero dentro ...",pase,telefono celular,pase,conexion a internet
2,005007,022,11,completa,casa independiente,adobe,"losetas, terrazos o similares",tejas,"red publica, dentro de la vivienda",pase,telefono celular,conexion a tv por cable o satelital,conexion a internet


## 2. Merge a Nivel Hogar

### 2.1 Inicializar el Merger

In [6]:
# Crear configuración para merge a nivel hogar
hogar_config = ModuleMergeConfig(merge_level=ModuleMergeLevel.HOGAR)

# Crear instancia del merger
merger = ENAHOGeoMerger(module_config=hogar_config, verbose=True)

print("✅ Merger inicializado para nivel hogar")
print(f"📋 Nivel de merge: {hogar_config.merge_level.value}")

[2025-10-18 12:56:06,505] [INFO] enaho_geo_merger: ENAHOGeoMerger inicializado correctamente


✅ Merger inicializado para nivel hogar
📋 Nivel de merge: hogar


### 2.2 Combinar Sumaria + Vivienda a Nivel Hogar

In [7]:
# Preparar diccionario de módulos a nivel hogar
modulos_hogar = {
    '34': df_sumaria,   # Módulo sumaria como base
    '01': df_vivienda   # Módulo vivienda
}

print("🔗 Realizando merge a nivel hogar...")
print(f"   - Módulo sumaria (34): {len(df_sumaria):,} hogares")
print(f"   - Módulo vivienda (01): {len(df_vivienda):,} hogares")

# Ejecutar merge (usa módulo 34 como base)
resultado_hogar = merger.merge_multiple_modules(
    modules_dict=modulos_hogar,
    base_module='34'  # Sumaria como módulo base
)

df_hogar_merged = resultado_hogar.merged_df

print(f"\n✅ Merge completado!")
print(f"   📊 Hogares resultantes: {len(df_hogar_merged):,}")
print(f"   📋 Columnas totales: {len(df_hogar_merged.columns)}")

[2025-10-18 12:56:08,889] [INFO] enaho_geo_merger: Iniciando merge de 2 módulos con base '34'
[2025-10-18 12:56:08,891] [INFO] enaho_geo_merger: Validando compatibilidad de 2 módulos
[2025-10-18 12:56:08,892] [INFO] enaho_geo_merger: Orden de merge: 34 → 01
[2025-10-18 12:56:08,894] [INFO] enaho_geo_merger: Agregando módulo 01
[2025-10-18 12:56:08,895] [INFO] enaho_geo_merger: 🔗 Iniciando merge: Módulo 34 + Módulo 01


🔗 Realizando merge a nivel hogar...
   - Módulo sumaria (34): 34,213 hogares
   - Módulo vivienda (01): 44,122 hogares


[2025-10-18 12:56:20,164] [INFO] enaho_geo_merger: ✅ Tipos armonizados para columna 'vivienda'
[2025-10-18 12:56:20,492] [INFO] enaho_geo_merger: ✅ Merge completado: 34213 registros finales (Calidad: 100.0%)
[2025-10-18 12:56:20,498] [INFO] enaho_geo_merger: Merge completado: 34213 registros finales (Calidad: 10000.0%)
[2025-10-18 12:56:20,499] [INFO] enaho_geo_merger: merge_multiple_modules completado en 11.61s con calidad total: 10000.0%



✅ Merge completado!
   📊 Hogares resultantes: 34,213
   📋 Columnas totales: 20


### 2.3 Analizar Resultado del Merge a Nivel Hogar

In [8]:
# Ver reporte del merge
print("📋 REPORTE DE MERGE A NIVEL HOGAR")
print("="*70)
for key, value in resultado_hogar.merge_report.items():
    print(f"   {key}: {value}")

print(f"\n📊 Calidad del merge: {resultado_hogar.quality_score:.2%}")
print(f"⚠️  Advertencias: {len(resultado_hogar.validation_warnings)}")

if resultado_hogar.validation_warnings:
    print("\n⚠️ Advertencias encontradas:")
    for warning in resultado_hogar.validation_warnings[:5]:  # Primeras 5
        print(f"   - {warning}")

📋 REPORTE DE MERGE A NIVEL HOGAR
   modules_merged: ['34', '01']
   base_module: 34
   merge_order: ['34', '01']
   total_records: 34213
   individual_reports: [{'module': '01', 'report': {'modules_merged': '34 + 01', 'merge_level': 'hogar', 'merge_strategy': 'coalesce', 'total_records': 34213, 'merge_statistics': {'both': 34213, 'left_only': 0, 'right_only': 0, 'total': 34213}, 'compatibility_info': {'compatible': True}, 'quality_score': 100.0, 'type_issues_fixed': 1, 'cardinality_warning': None}, 'quality_score': 100.0}]
   elapsed_time: 11.610004901885986
   timestamp: 2025-10-18T12:56:20.499105

📊 Calidad del merge: 10000.00%
⚠️  Advertencias: 3

⚠️ Advertencias encontradas:
   - Sumaria: 1 hogares con número de miembros inválido
   - Sumaria: 1055 hogares con gastos muy superiores a ingresos
   - Tipo incompatible en vivienda (category vs category)


### 2.4 Explorar Datos Combinados a Nivel Hogar

In [9]:
# Ver columnas disponibles
print("📋 Columnas del dataset combinado:")
print(f"\nColumnas de Sumaria ({len([c for c in df_hogar_merged.columns if c in df_sumaria.columns])}):")
print([c for c in df_hogar_merged.columns if c in df_sumaria.columns])

print(f"\nColumnas de Vivienda ({len([c for c in df_hogar_merged.columns if c in df_vivienda.columns and c not in df_sumaria.columns])}):")
print([c for c in df_hogar_merged.columns if c in df_vivienda.columns and c not in df_sumaria.columns])

# Muestra de datos combinados
print("\n📊 Muestra de datos combinados:")
display(df_hogar_merged[[
    'ubigeo', 'pobreza', 'mieperho', 'gashog2d', 'inghog2d',  # De sumaria
    'p101', 'p110',  'p1143'  # De vivienda
]].head())

📋 Columnas del dataset combinado:

Columnas de Sumaria (10):
['conglome', 'vivienda', 'hogar', 'ubigeo', 'dominio', 'estrato', 'mieperho', 'gashog2d', 'inghog2d', 'pobreza']

Columnas de Vivienda (10):
['result', 'p101', 'p102', 'p103', 'p103a', 'p110', 'p1141', 'p1142', 'p1143', 'p1144']

📊 Muestra de datos combinados:


,ubigeo,pobreza,mieperho,gashog2d,inghog2d,p101,p110,p1143
0,010201,no pobre,2,36949.523438,34606.441406,casa independiente,"red publica, dentro de la vivienda",conexion a tv por cable o satelital
1,010201,no pobre,2,48161.250000,82028.218750,casa independiente,"red publica, dentro de la vivienda",conexion a tv por cable o satelital
2,010201,no pobre,4,27575.878906,31791.029297,casa independiente,"red publica, dentro de la vivienda",pase
3,010201,no pobre,3,16161.043945,24028.867188,casa independiente,"red publica, dentro de la vivienda",conexion a tv por cable o satelital
4,010201,pobre no extremo,11,30606.212891,46056.109375,casa independiente,"red publica, dentro de la vivienda",conexion a tv por cable o satelital


### 2.5 Análisis Exploratorio del Dataset Combinado

In [10]:
# Análisis de pobreza por tipo de vivienda
print("📊 Análisis: Pobreza por Tipo de Vivienda")
print("="*70)

crosstab_pobreza_vivienda = pd.crosstab(
    df_hogar_merged['p101'],  # Tipo de vivienda
    df_hogar_merged['pobreza'],  # Condición de pobreza
    margins=True,
    margins_name='Total'
)

display(crosstab_pobreza_vivienda)

# Análisis de servicios básicos por condición de pobreza
print("\n💡 Acceso a Televisión por Condición de Pobreza:")
internet_pobreza = pd.crosstab(
    df_hogar_merged['pobreza'],
    df_hogar_merged['p1143'],  # Internet
    normalize='index'
) * 100

display(internet_pobreza.round(2))

📊 Análisis: Pobreza por Tipo de Vivienda


pobreza,no pobre,pobre extremo,pobre no extremo,Total
p101,,,,
casa independiente,23951,1418,5037,30406
choza o cabaña,142,58,100,300
departamento en edificio,1508,2,66,1576
local no destinado para habitacion humana,1,0,0,1
"vivienda en casa de vecindad (callejon, solar o corralon)",1053,17,204,1274
vivienda en quinta,250,6,35,291
vivienda improvisada,6,0,0,6
Total,26911,1501,5442,33854



💡 Acceso a Televisión por Condición de Pobreza:


p1143,conexion a tv por cable o satelital,pase
pobreza,,
no pobre,34.40,65.60
pobre extremo,6.19,93.81
pobre no extremo,16.15,83.85


## 3. Merge a Nivel Persona

### 3.1 Cargar Datos a Nivel Persona

In [19]:
import glob
import os

# Descargar Módulo 02 (Características de Miembros)
#print("📥 Descargando Módulo 02 (Características)...")
#downloader.download(modules=["02"], years=[str(year)], output_dir=".enaho_cache", decompress=True)

# Encontrar y leer archivo
dta_files_02 = glob.glob(f".enaho_cache/modulo_02_{year}/*.dta")
if dta_files_02:
    file_path_02 = dta_files_02[0]
    reader_02 = ENAHOLocalReader(file_path=file_path_02, verbose=False)
    df_caracteristicas, _ = reader_02.read_data(columns=[
        'conglome', 'vivienda', 'hogar', 'codperso',  # Identificadores
        'p203',      # Parentesco
        'p207',      # Sexo
        'p208a',      # Edad
        'p209',     # estado civil
    ])
    print(f"✅ Módulo 02 cargado: {len(df_caracteristicas):,} personas")
else:
    raise FileNotFoundError("No se encontró archivo .dta para módulo 02")

# Descargar Módulo 05 (Empleo e Ingresos)
#print("\n📥 Descargando Módulo 05 (Empleo)...")
#downloader.download(modules=["05"], years=[str(year)], output_dir=".enaho_cache", decompress=True)

# Encontrar y leer archivo
dta_files_05 = glob.glob(f".enaho_cache/modulo_05_{year}/*.dta")
if dta_files_05:
    file_path_05 = dta_files_05[0]
    reader_05 = ENAHOLocalReader(file_path=file_path_05, verbose=False)
    df_empleo, _ = reader_05.read_data(columns=[
        'conglome', 'vivienda', 'hogar', 'codperso',  # Identificadores
        'ocu500',    # Condición de actividad
        'p506',      # Ocupación principal
        'p507',      # Categoría ocupacional
        'p511a',     # Horas trabajadas
        'i524a1',    # Ingreso trabajo dependiente
        'i530a',     # Ingreso trabajo independiente
        'p523',      # Tiene contrato escrito
        'p524a1'     # Tipo de contrato
    ])
    print(f"✅ Módulo 05 cargado: {len(df_empleo):,} personas")
else:
    raise FileNotFoundError("No se encontró archivo .dta para módulo 05")

# Mostrar primeras filas
print("\n📊 Muestra de datos:")
print("\nMódulo 02 (Características):")
display(df_caracteristicas.head(3))

print("\nMódulo 05 (Empleo):")
display(df_empleo.head(3))

✅ Módulo 02 cargado: 121,253 personas
✅ Módulo 05 cargado: 87,661 personas

📊 Muestra de datos:

Módulo 02 (Características):


,conglome,vivienda,hogar,codperso,p203,p207,p208a,p209
0,005030,008,11,01,jefe/jefa,hombre,85,casado(a)
1,005030,008,11,02,esposo(a)/compañero(a),mujer,75,casado(a)
2,005030,008,11,03,otros no parientes,mujer,53,soltero(a)



Módulo 05 (Empleo):


,conglome,vivienda,hogar,codperso,p506,p507,p511a,p523,p524a1,i530a,i524a1,ocu500
0,005007,003,11,01,instituto de enseñanza superior,empleado,"contrato indefinido, nombrado, permanente",mensual,5000,NaN,63060.0,ocupado
1,005007,012,11,01,servicio de comedor en trenes,obrero,sin contrato,mensual,800,NaN,10090.0,ocupado
2,005007,022,11,01,administracion y aplicacion de impuestos,empleado,"contrato indefinido, nombrado, permanente",mensual,1660,NaN,20936.0,ocupado


### 3.2 Combinar Características + Empleo a Nivel Persona

In [41]:
# Preparar diccionario de módulos a nivel persona
modulos_persona = {
    '02': df_caracteristicas,  # Módulo características como base
    '05': df_empleo
}

print("🔗 Realizando merge a nivel persona...")
print(f"   - Módulo características (02): {len(df_caracteristicas):,} personas")
print(f"   - Módulo empleo (05): {len(df_empleo):,} personas")

# Crear configuración para nivel persona
persona_config = ModuleMergeConfig(merge_level=ModuleMergeLevel.PERSONA)

# Crear nuevo merger para nivel persona
merger_persona = ENAHOGeoMerger(module_config=persona_config, verbose=True)

# Ejecutar merge (usa módulo 02 como base)
resultado_persona = merger_persona.merge_multiple_modules(
    modules_dict=modulos_persona,
    base_module='02'  # Características como módulo base
)

df_persona_merged = resultado_persona.merged_df

print(f"\n✅ Merge completado!")
print(f"   📊 Personas resultantes: {len(df_persona_merged):,}")
print(f"   📋 Columnas totales: {len(df_persona_merged.columns)}")

[2025-10-18 13:34:06,137] [INFO] enaho_geo_merger: ENAHOGeoMerger inicializado correctamente
[2025-10-18 13:34:06,139] [INFO] enaho_geo_merger: Iniciando merge de 2 módulos con base '02'
[2025-10-18 13:34:06,140] [INFO] enaho_geo_merger: Validando compatibilidad de 2 módulos
[2025-10-18 13:34:06,141] [INFO] enaho_geo_merger: Orden de merge: 02 → 05
[2025-10-18 13:34:06,143] [INFO] enaho_geo_merger: Agregando módulo 05
[2025-10-18 13:34:06,144] [INFO] enaho_geo_merger: 🔗 Iniciando merge: Módulo 02 + Módulo 05


🔗 Realizando merge a nivel persona...
   - Módulo características (02): 121,253 personas
   - Módulo empleo (05): 87,661 personas


[2025-10-18 13:34:16,293] [INFO] enaho_geo_merger: ✅ Tipos armonizados para columna 'codperso'
[2025-10-18 13:34:16,998] [INFO] enaho_geo_merger: ✅ Merge completado: 121253 registros finales (Calidad: 66.8%)
[2025-10-18 13:34:17,016] [INFO] enaho_geo_merger: Merge completado: 121253 registros finales (Calidad: 6675.5%)
[2025-10-18 13:34:17,018] [INFO] enaho_geo_merger: merge_multiple_modules completado en 10.88s con calidad total: 6675.5%



✅ Merge completado!
   📊 Personas resultantes: 121,253
   📋 Columnas totales: 16


### 3.3 Analizar Resultado del Merge a Nivel Persona

In [21]:
# Ver reporte del merge
print("📋 REPORTE DE MERGE A NIVEL PERSONA")
print("="*70)
for key, value in resultado_persona.merge_report.items():
    print(f"   {key}: {value}")

print(f"\n📊 Calidad del merge: {resultado_persona.quality_score:.2%}")
print(f"⚠️  Advertencias: {len(resultado_persona.validation_warnings)}")

# Analizar registros no emparejados
print(f"\n📈 Personas sin datos de empleo: {resultado_persona.unmatched_right:,}")
print("   (Esto es normal: niños, personas que no participan en la fuerza laboral, etc.)")

📋 REPORTE DE MERGE A NIVEL PERSONA
   modules_merged: ['02', '05']
   base_module: 02
   merge_order: ['02', '05']
   total_records: 121253
   individual_reports: [{'module': '05', 'report': {'modules_merged': '02 + 05', 'merge_level': 'persona', 'merge_strategy': 'coalesce', 'total_records': 121253, 'merge_statistics': {'both': 87661, 'left_only': 33592, 'right_only': 0, 'total': 121253}, 'compatibility_info': {'compatible': True}, 'quality_score': 66.75513183178973, 'type_issues_fixed': 1, 'cardinality_warning': None}, 'quality_score': 66.75513183178973}]
   elapsed_time: 11.082005739212036
   timestamp: 2025-10-18T13:13:15.673731

📊 Calidad del merge: 6675.51%
⚠️  Advertencias: 4

📈 Personas sin datos de empleo: 0
   (Esto es normal: niños, personas que no participan en la fuerza laboral, etc.)


### 3.4 Explorar Datos Combinados a Nivel Persona

In [26]:
# Ver muestra de datos combinados
print("📊 Muestra de datos combinados a nivel persona:")
display(df_persona_merged[[
    'codperso', 'p207', 'p208a', 'p209',  # Características
    'ocu500', 'p507', 'p511a', 'i524a1'   # Empleo
]].head(10))


📊 Muestra de datos combinados a nivel persona:


,codperso,p207,p208a,p209,ocu500,p507,p511a,i524a1
0,01,hombre,85,casado(a),no pea,NaN,NaN,NaN
1,02,mujer,75,casado(a),no pea,NaN,NaN,NaN
2,03,mujer,53,soltero(a),NaN,NaN,NaN,NaN
3,01,mujer,77,viudo(a),ocupado,trabajador independiente,NaN,NaN
4,02,hombre,55,separado(a),ocupado,trabajador familiar no remunerado,sin contrato,NaN
5,01,mujer,40,soltero(a),ocupado,trabajador independiente,NaN,NaN
6,02,mujer,70,viudo(a),ocupado,trabajador familiar no remunerado,sin contrato,NaN
7,03,mujer,37,separado(a),no pea,NaN,NaN,NaN
8,04,hombre,14,soltero(a),ocupado,trabajador familiar no remunerado,sin contrato,NaN
9,05,mujer,39,conviviente,NaN,NaN,NaN,NaN


### 3.5 Análisis: Empleo por Nivel Educativo y Sexo

In [45]:
# Filtrar personas en edad de trabajar (14+ años) - versión robusta
df_pet = df_persona_merged[pd.to_numeric(df_persona_merged['p208a'], errors='coerce') >= 14].copy()

print(f"📊 Personas en Edad de Trabajar (PET): {len(df_pet):,}")

# El resto de tu código continúa igual...
# Tasa de actividad por sexo
print("\n💼 Condición de Actividad por Sexo:")
actividad_sexo = pd.crosstab(
    df_pet['p207'],  # Sexo
    df_pet['ocu500'].notna(),  # Tiene datos de empleo
    normalize='index'
) * 100

actividad_sexo.columns = ['Sin datos empleo', 'Con datos empleo']
display(actividad_sexo.round(2))

# Ingreso promedio por nivel educativo
#print("\n💰 Ingreso Promedio por Nivel Educativo:")
#df_pet['ingreso_total'] = df_pet['i524a1'].fillna(0) + df_pet['i530a'].fillna(0)
#ingreso_educacion = df_pet.groupby('p208a')['ingreso_total'].agg(['mean', 'median', 'count'])
#ingreso_educacion.columns = ['Promedio', 'Mediana', 'N']
#display(ingreso_educacion.round(2))

📊 Personas en Edad de Trabajar (PET): 90,227

💼 Condición de Actividad por Sexo:


,Sin datos empleo,Con datos empleo
p207,,
hombre,2.78,97.22
mujer,2.90,97.10


In [47]:
# Verificar la columna de edad
print("🔍 Diagnóstico de la columna 'p208a' (edad):")
print(f"Tipo de datos: {df_persona_merged['p208a'].dtype}")
print(f"Valores únicos: {df_persona_merged['p208a'].unique()}")
print(f"Valores nulos: {df_persona_merged['p208a'].isna().sum()}")
print(f"Forma del DataFrame original: {df_persona_merged.shape}")

# Convertir a numérico y verificar
edades_numericas = pd.to_numeric(df_persona_merged['p208a'], errors='coerce')
print(f"\n📊 Estadísticas después de conversión numérica:")
print(f"Valores mínimos: {edades_numericas.min()}")
print(f"Valores máximos: {edades_numericas.max()}")
print(f"Valores nulos después de conversión: {edades_numericas.isna().sum()}")

🔍 Diagnóstico de la columna 'p208a' (edad):
Tipo de datos: category
Valores únicos: [85, 75, 53, 77, 55, ..., 93, 95, 92, 94, 97]
Length: 100
Categories (99, int64): [0, 1, 2, 3, ..., 95, 96, 97, 98]
Valores nulos: 3126
Forma del DataFrame original: (121253, 16)

📊 Estadísticas después de conversión numérica:
Valores mínimos: 0.0
Valores máximos: 98.0
Valores nulos después de conversión: 3126


In [48]:
# Convertir a numérico y filtrar correctamente

edades_numericas = pd.to_numeric(df_persona_merged['p208a'], errors='coerce')

# Filtrar personas en edad de trabajar (14+ años)
df_pet = df_persona_merged[edades_numericas >= 14].copy()

print(f"📊 Personas en Edad de Trabajar (PET): {len(df_pet):,}")

# Si sigue siendo 0, investigamos más
if len(df_pet) == 0:
    print("\n⚠️  ¡Advertencia! El DataFrame filtrado está vacío.")
    print("Posibles causas:")
    print("1. La columna 'p205' no contiene valores numéricos válidos")
    print("2. No hay personas con 14+ años en los datos")
    print("3. La conversión numérica está fallando")

    # Verificar algunos registros específicos
    print("\n🔍 Muestra de registros originales:")
    print(df_persona_merged[['p205']].head(10))

    # Crear un DataFrame temporal para continuar el análisis
    df_pet = df_persona_merged.copy()
    print("⚠️  Continuando con DataFrame sin filtrar para análisis...")

📊 Personas en Edad de Trabajar (PET): 90,227


## 4. Merge Multi-Nivel: Combinar Datos de Hogar y Persona

### 4.1 Agregar Datos de Hogar a las Personas

In [50]:
# Combinar datos de persona con datos del hogar
print("🔗 Combinando datos de persona con características del hogar...")

# Llaves de hogar
id_hogar = ['conglome', 'vivienda', 'hogar']

# Merge de personas con hogares
df_persona_hogar = df_persona_merged.merge(
    df_hogar_merged[id_hogar + ['pobreza', 'gashog2d', 'inghog2d', 'ubigeo', 'mieperho', 'p110', ]],
    on=id_hogar,
    how='left'
)

print(f"✅ Merge completado!")
print(f"   📊 Personas con datos de hogar: {len(df_persona_hogar):,}")
print(f"   📋 Columnas totales: {len(df_persona_hogar.columns)}")

# Muestra
print("\n📊 Muestra del dataset completo:")
display(df_persona_hogar[[
    'codperso', 'p208a', 'p209', 'p207',  # Persona
    'ocu500', 'i524a1',                    # Empleo
    'pobreza', 'gashog2d', 'p110'         # Hogar
]].head())

🔗 Combinando datos de persona con características del hogar...
✅ Merge completado!
   📊 Personas con datos de hogar: 121,253
   📋 Columnas totales: 22

📊 Muestra del dataset completo:


,codperso,p208a,p209,p207,ocu500,i524a1,pobreza,gashog2d,p110
0,01,85,casado(a),hombre,no pea,NaN,no pobre,36949.523438,"red publica, dentro de la vivienda"
1,02,75,casado(a),mujer,no pea,NaN,no pobre,36949.523438,"red publica, dentro de la vivienda"
2,03,53,soltero(a),mujer,NaN,NaN,no pobre,36949.523438,"red publica, dentro de la vivienda"
3,01,77,viudo(a),mujer,ocupado,NaN,no pobre,48161.250000,"red publica, dentro de la vivienda"
4,02,55,separado(a),hombre,ocupado,NaN,no pobre,48161.250000,"red publica, dentro de la vivienda"


### 4.2 Análisis Integrado: Empleo y Pobreza

In [57]:
edades_numericas = pd.to_numeric(df_persona_hogar['p208a'], errors='coerce')


In [63]:

# Filtrar PET
df_pet_hogar = df_persona_hogar[edades_numericas >= 14].copy()

# Crear indicador de ocupación
df_pet_hogar['ocupado'] = df_pet_hogar['ocu500'].notna()

# Tasa de ocupación por condición de pobreza
print("📊 Tasa de Ocupación por Condición de Pobreza:")
ocupacion_pobreza = pd.crosstab(
    df_pet_hogar['pobreza'],
    df_pet_hogar['ocupado'],
    normalize='index'
) * 100

ocupacion_pobreza.columns = ['No ocupado', 'Ocupado']
display(ocupacion_pobreza.round(2))

# Ingreso promedio por pobreza y sexo
print("\n💰 Ingreso Promedio por Pobreza y Sexo:")
df_pet_hogar['ingreso_total'] = df_pet_hogar['i524a1'].fillna(0) + df_pet_hogar['i530a'].fillna(0)
ingreso_pivot = df_pet_hogar.pivot_table(
    values='ingreso_total',
    index='pobreza',
    columns='p207',
    aggfunc='mean'
)
display(ingreso_pivot.round(2))

📊 Tasa de Ocupación por Condición de Pobreza:


,No ocupado,Ocupado
pobreza,,
no pobre,3.06,96.94
pobre extremo,2.07,97.93
pobre no extremo,2.10,97.90



💰 Ingreso Promedio por Pobreza y Sexo:


p207,hombre,mujer
pobreza,,
no pobre,12718.080078,6648.379883
pobre extremo,3575.060059,1094.569946
pobre no extremo,6166.549805,2263.260010


## 5. Validación de Calidad de Merge

### 5.1 Verificar Integridad Referencial

In [64]:
# Verificar que no se perdieron hogares en el merge
print("🔍 Verificación de Integridad:")
print("="*70)

print("\n📊 A NIVEL HOGAR:")
print(f"   Hogares en Sumaria (34): {len(df_sumaria):,}")
print(f"   Hogares en Vivienda (01): {len(df_vivienda):,}")
print(f"   Hogares merged: {len(df_hogar_merged):,}")
print(f"   Pérdida: {len(df_sumaria) - len(df_hogar_merged):,} hogares")

print("\n📊 A NIVEL PERSONA:")
print(f"   Personas en Características (02): {len(df_caracteristicas):,}")
print(f"   Personas en Empleo (05): {len(df_empleo):,}")
print(f"   Personas merged: {len(df_persona_merged):,}")
print(f"   Diferencia: {len(df_caracteristicas) - len(df_persona_merged):,} personas")

🔍 Verificación de Integridad:

📊 A NIVEL HOGAR:
   Hogares en Sumaria (34): 34,213
   Hogares en Vivienda (01): 44,122
   Hogares merged: 34,213
   Pérdida: 0 hogares

📊 A NIVEL PERSONA:
   Personas en Características (02): 121,253
   Personas en Empleo (05): 87,661
   Personas merged: 121,253
   Diferencia: 0 personas


### 5.2 Análisis de Valores Faltantes Post-Merge

In [65]:
# Columnas con mayor porcentaje de valores faltantes en el merge de personas
print("📊 Columnas con Valores Faltantes (Top 10):")
missing_pct = (df_persona_merged.isna().sum() / len(df_persona_merged) * 100).sort_values(ascending=False)
display(missing_pct.head(10).round(2))

print("\n💡 Nota: Es normal tener valores faltantes en columnas de empleo")
print("   para personas que no forman parte de la PEA (menores, estudiantes, etc.)")

📊 Columnas con Valores Faltantes (Top 10):


p524a1    79.10
p523      78.85
i524a1    78.85
i530a     77.35
p511a     69.06
p506      46.20
p507      46.20
ocu500    27.70
p209      21.65
p207       2.58
dtype: float64


💡 Nota: Es normal tener valores faltantes en columnas de empleo
   para personas que no forman parte de la PEA (menores, estudiantes, etc.)


## 6. Guardar Resultados

In [66]:
# Guardar datasets combinados
output_dir = "output"
import os
os.makedirs(output_dir, exist_ok=True)

# Guardar datos de hogar
df_hogar_merged.to_csv(f"{output_dir}/enaho_{year}_hogar_merged.csv", index=False)
print(f"✅ Datos de hogar guardados en: {output_dir}/enaho_{year}_hogar_merged.csv")

# Guardar datos de persona
df_persona_merged.to_csv(f"{output_dir}/enaho_{year}_persona_merged.csv", index=False)
print(f"✅ Datos de persona guardados en: {output_dir}/enaho_{year}_persona_merged.csv")

# Guardar datos combinados persona-hogar
df_persona_hogar.to_csv(f"{output_dir}/enaho_{year}_persona_hogar.csv", index=False)
print(f"✅ Datos persona-hogar guardados en: {output_dir}/enaho_{year}_persona_hogar.csv")

✅ Datos de hogar guardados en: output/enaho_2022_hogar_merged.csv
✅ Datos de persona guardados en: output/enaho_2022_persona_merged.csv
✅ Datos persona-hogar guardados en: output/enaho_2022_persona_hogar.csv


## Resumen

En este tutorial aprendiste a:

1. ✅ Combinar módulos ENAHO a nivel hogar (Sumaria + Vivienda)
2. ✅ Combinar módulos a nivel persona (Características + Empleo)
3. ✅ Realizar merges multi-nivel (Persona + Hogar)
4. ✅ Interpretar reportes de calidad de merge
5. ✅ Validar integridad referencial
6. ✅ Analizar valores faltantes
7. ✅ Realizar análisis exploratorios con datos combinados

### Próximos Pasos

- **Tutorial 03**: Análisis de valores faltantes con `null_analysis`
- **Tutorial 04**: Pipeline completo de análisis